## *Special Token Operations
#### 04.4 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-09 (started 2021-11-28)


### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()['last-round']

27733155

## Modifications of an ASA
Most properties of an ASA can **never** be modified
* Name and unit name
* Decimals
* Total supply

Using the special roles, it is however possible to ...
* Freeze/unfreeze ASA assets of a specific address
* Claim ASA back from a specific address
* Destroy an ASA
* Change the special roles


### First cretate Tempcoin
To study the whole life of an ASA, we create a new one, called `Tempcoin`, with the symbol `TEMP` (for temporary), with the following roles:

* **Manager** = Alice
* **Reserve** = Bob
* **Freeze** = Charlie
* **Clawback** = Dina (this will change later to Alice)

In [19]:
# Step 1: Prepare
sp = algod_client.suggested_params()
token_supply = 100                               # Token supply
token_decimals =  1                              # How many digits after the comma?
token_total = token_supply * 10**token_decimals  # Specify SMALLER unit ("cents")

token_name  = "Pyrys Tempcoin"                  # <----- YOUR NAME HERE
token_url   = "en.wikipedia.org/wiki/Temporaryy"  # <----- CHANGE if you want to
token_unitname = "TEMPYRY"

# Step 2: Asset creation tansaction
txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                                     # Network parameters
    total=token_total,                         # Token supply in SMALL unit
    decimals=token_decimals,
    default_frozen=False,                      
    unit_name=token_unitname,                       
    asset_name=token_name,
    url=token_url,
    manager=Alice['public'],                   # Special roles
    reserve=Bob['public'],
    freeze=Charlie['public'],
    clawback=Dina['public'], 
)

# Step 3+4: Sign and send
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

# Step 5: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

WPPZJWIATRZPDSQAVS4Y2YZMAGJT7QBZ446TJQEO47GWKZKLZ3CA
Current round is  27733295.
Waiting for round 27733295 to finish.
Waiting for round 27733296 to finish.
Transaction WPPZJWIATRZPDSQAVS4Y2YZMAGJT7QBZ446TJQEO47GWKZKLZ3CA confirmed in round 27733297.


In [20]:
# Step 6: obtain asset_id
TEMP_id = txinfo['asset-index']
print(TEMP_id)

159173586


In [21]:
asset_holdings_df(algod_client,MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,3.425250e+01,ALGO,0,Algorand,6
1,1.100000e+02,USDC,10458941,USDC,6
2,0.000000e+00,DRZY,12887013,Drizzy,1
3,9.000000e+02,HUSKEN,159159432,Husky Token,2
4,1.000000e+02,DWSC,159159534,Daniels WSC coin,2
5,9.979900e+09,Beer,159171974,Beer Coin,2
6,1.670000e+01,TEMP,159173248,Peters Tempcoin,1
7,1.000000e+02,TEMPYRY,159173545,Pyrys Tempcoin,1
8,1.000000e+02,TEMPYRY,159173586,Pyrys Tempcoin,1


## The Reserve address

##### Step 1: Check in Algoexplorer
* Total supply
* Circulating supply

In [23]:
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

https://testnet.algoexplorer.io/asset/159173586


##### Step 2: Transfer 50% to the reserve address
* So that they are shown as "not yet minted"

##### Step 2a: Bob opts in
* Even though Bob is the reserve, he has to opt in

In [24]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(0)                              # <------ opt-in = 0 transaction

txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- From Bo ...
    sp=sp,
    receiver=Bob['public'],               # <------- ... to Bob
    amt=amt,
    index=TEMP_id)                        # <----- Correct index TEMP_id

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

EJKAHN7JDG24GRM7ROM3CNLAOGHYHM73CVJLZK3POXDIGZKQEFSQ
Current round is  27733305.
Waiting for round 27733305 to finish.
Waiting for round 27733306 to finish.
Transaction EJKAHN7JDG24GRM7ROM3CNLAOGHYHM73CVJLZK3POXDIGZKQEFSQ confirmed in round 27733307.


##### Step 2b: Creator (=MyAlgo) transfers to the reserve (= Bob)

In [25]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(token_total / 3)                # <------- half of the holdings
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Bob['public'],               # <------- Bob is the reserve
    amt=amt,
    index=TEMP_id)                        # <------- Correct index TEMP_id

# Step 2+3: sign and send
stxn = txn.sign(MyAlgo['private'])         # <----- Signed by the creator of the asset
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: wait for confirmation
wait_for_confirmation(algod_client, txid)

5UFFNQVLYEURZ5IHN4MWPA7ZSEB6KFGX2UNFDTMWSIYLTBEV2C5Q
Current round is  27733308.
Waiting for round 27733308 to finish.
Waiting for round 27733309 to finish.
Transaction 5UFFNQVLYEURZ5IHN4MWPA7ZSEB6KFGX2UNFDTMWSIYLTBEV2C5Q confirmed in round 27733310.


{'confirmed-round': 27733310,
 'pool-error': '',
 'txn': {'sig': 'omszZ1JebxMIgkK3ZpiZbfc6qZxK4DhY3qzurVV4yDBV5OznqnKpMiM9rD/x6w9AQhxGjkQtn3kjiPMBZk06DQ==',
  'txn': {'aamt': 333,
   'arcv': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM',
   'fee': 1000,
   'fv': 27733307,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 27734307,
   'snd': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'type': 'axfer',
   'xaid': 159173586}}}

In [26]:
asset_holdings_df(algod_client,Bob['public'])

,amount,unit,asset-id,name,decimals
0,2.385000e+00,ALGO,0,Algorand,6
1,2.000000e+01,USDC,10458941,USDC,6
2,2.010002e+07,Beer,159171974,Beer Coin,2
3,8.330000e+01,TEMP,159173248,Peters Tempcoin,1
4,3.330000e+01,TEMPYRY,159173586,Pyrys Tempcoin,1


##### Step 3: Check "Circulating Supply" in Algoexplorer

In [27]:
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

https://testnet.algoexplorer.io/asset/159173586


## Clawback a transaction
* Normally, only the owner of a token could authorize a transfer of his tokens ... except for the clawback address `Dina`.
* She claws back 25 tokens from Bob by specifying Bob as `revocation_target`
* The clawed back coins are sent to the creator
* No opt-in necessary for clawback, because she does not receive any coins

##### Step 1: Clawback transaction
* using `revocation_target`

In [28]:
sp = algod_client.suggested_params()

# Step 1: Create clawback transaction
txn = AssetTransferTxn(
    sender=Dina['public'],                  # <---- Clawback = Dina is sender of transaction
    sp=sp,
    receiver=MyAlgo["public"],              # <---- Money goes back to MyAlgo
    amt=25*10,                              # <---- Amount in SMALL units
    index=TEMP_id,
    revocation_target=Bob['public']         # <---- Take the money out of Bob's account
    )

# Step 2: sign by clawback
stxn = txn.sign(Dina['private'])            # <---- Signed by Clawback = Dina

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for the transaction to be confirmed
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27733316.
Waiting for round 27733316 to finish.
Waiting for round 27733317 to finish.
Transaction 4QLWTBZVU4SJJ25APEKCD273SVMP2ZM46BKGFRRQANCWIJEI6M4Q confirmed in round 27733318.


##### Step 2: Check Bob's holdings
* He now has 25 fewer coins

In [29]:
# Check Bob's asset holdings
asset_holdings_df(algod_client,Bob['public'])

,amount,unit,asset-id,name,decimals
0,2.385000e+00,ALGO,0,Algorand,6
1,2.000000e+01,USDC,10458941,USDC,6
2,2.010002e+07,Beer,159171974,Beer Coin,2
3,8.330000e+01,TEMP,159173248,Peters Tempcoin,1
4,8.300000e+00,TEMPYRY,159173586,Pyrys Tempcoin,1


##### Step 3: Where did the 25 coins go?
* They always go to the creator (not the the clawback)

In [30]:
asset_holdings_df(algod_client,MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,3.425150e+01,ALGO,0,Algorand,6
1,1.100000e+02,USDC,10458941,USDC,6
2,0.000000e+00,DRZY,12887013,Drizzy,1
3,9.000000e+02,HUSKEN,159159432,Husky Token,2
4,1.000000e+02,DWSC,159159534,Daniels WSC coin,2
5,9.979900e+09,Beer,159171974,Beer Coin,2
6,1.670000e+01,TEMP,159173248,Peters Tempcoin,1
7,1.000000e+02,TEMPYRY,159173545,Pyrys Tempcoin,1
8,9.170000e+01,TEMPYRY,159173586,Pyrys Tempcoin,1


## Reconfigure the asset
* A reconfiguration is changing any of the four special roles
* The **Manager** (Alice) can reconfigure the asset. 
* In a reconfiguration, all four roles must again be specified
* No opt-in is necessary for reconfigure, because no coins are receved

**Example**
* Alice removes the clawback role from Dina and assigns it to herself
* Similar to `AssetConfigTxn` as when creating an asset

In [31]:
sp = algod_client.suggested_params()

# Step 1: Create Asset reconfiguration tansaction
txn = AssetConfigTxn(
    sender=Alice['public'],                    # Manager
    sp=sp,                                     # Network parameters
    index=TEMP_id,                             # <------------ MUST specify the asset ID
    manager=Alice['public'],                   # Old role, repeated
    reserve=Bob['public'],                     # Old role, repeated
    freeze=Charlie['public'],                  # Old role, repeated
    clawback=Alice['public'],                  # <-------- New responsibiliy
)

# Step 2+3: Sign and send
stxn = txn.sign(Alice['private'])              # <---- Signed by the manager!
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

7HKUK3BCOC2LMUN5CP3JIITOK7DUWO47B2YT7GWGZREHOIFZOLXQ
Current round is  27733340.
Waiting for round 27733340 to finish.
Waiting for round 27733341 to finish.
Transaction 7HKUK3BCOC2LMUN5CP3JIITOK7DUWO47B2YT7GWGZREHOIFZOLXQ confirmed in round 27733342.


In [32]:
# Open in Algoexplorer: manager and clawback are now the same
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

https://testnet.algoexplorer.io/asset/159173586


**TRY THIS** If Dina tries to clawback the other half of Bob's holdings, it will not work

## Freeze assets

#### Charlie freezes Bob's assets
* The freeze role (= Charlie) freezes the asset holdings of Bob.
* New transaction type `AssetFreezeTxn`
* No opt-in necessary for freezing

In [51]:
# Step 1: Prepare
sp = algod_client.suggested_params()

# Step 2: Asset freeze tansaction
txn = AssetFreezeTxn(
    sender=Charlie['public'],                # <---- Must be initiated by freeze address
    sp=sp,
    index=TEMP_id,
    target=Bob["public"],                    # <---- This is the address that we want to freeze  
    new_freeze_state=True   
    )

# Step 3: Sign and send
stxn = txn.sign(Charlie['private'])          # <---- Must be signed by the freeze role(=Charlie)
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

MAQ5WS7WWFXEDR7FY6Y6CNW5ZPU62JPT5T2IUHXBMTBPRCTDJAKA
Current round is  27733936.
Waiting for round 27733936 to finish.
Waiting for round 27733937 to finish.
Transaction MAQ5WS7WWFXEDR7FY6Y6CNW5ZPU62JPT5T2IUHXBMTBPRCTDJAKA confirmed in round 27733938.


#### Bob wants to spend some TEMP token
* Bob tries to send some TEMP token to Alice, but he cannot do it
* Only his TEMP token are frozen

In [52]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(5 * 10)    

txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- Bob wants to send ...
    sp=sp,
    receiver=Alice['public'],             # <------- ... to Alice  
    amt=amt,
    index=TEMP_id)                        # <----- asset_id for TEMP token

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob

try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("frozen" in str(err)):                # check for specific type of error
        print("Asset is frozen")         
    txid = None
    
# Step 4: Wait for confirmation
# There is no step 4 here, because we already obtain an error on step 3

TransactionPool.Remember: transaction G7TKVASQHMPKGZD3I5QO476SYPKSBB5YK2HSSZELNRWIAEWB7H4A: asset 159173586 frozen in MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM
Asset is frozen


#### Exercise: unfreezeing Bobs tokens
* How can we unfreeze Bobs tokens? 
* The operation is similar to freezing, except for ...

In [55]:
sp = algod_client.suggested_params()

# Step 2: Asset freeze tansaction
txn = AssetFreezeTxn(
    sender=Charlie['public'],                # <---- Must be initiated by freeze address
    sp=sp,
    index=TEMP_id,
    target=Bob["public"],                    # <---- This is the address that we want to freeze  
    new_freeze_state=True   
    )

# Step 3: Sign and send
stxn = txn.sign(Charlie['private'])          # <---- Must be signed by the freeze role(=Charlie)
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)



GK64WXKUJTOJI6VOC6KMQMZFLUUDJZT7P55OXIYEUKPRJ262VONQ
Current round is  27733954.
Waiting for round 27733954 to finish.
Waiting for round 27733955 to finish.
Waiting for round 27733956 to finish.
Transaction GK64WXKUJTOJI6VOC6KMQMZFLUUDJZT7P55OXIYEUKPRJ262VONQ confirmed in round 27733957.


## Destroy asset
* An asset can only be destroyed, if all coins are back in the creator's account (MyAlgo). 
* Then the manager (Alice) can destroy the asset.

#### Return all TEMP coins to MyAlgo and opt out
* Use `AssetTransferTxn` to send back coins
* Addtionally we have to opt out, using `close_assets_to`
    * *Closing* means sending the remaining balance to a specific address

In [56]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(25 * 10)    

txn = AssetTransferTxn(
    sender=Bob['public'],                
    sp=sp,
    receiver=MyAlgo['public'],            
    amt=amt,
    index=TEMP_id,                        
    close_assets_to=MyAlgo['public']      # <------- Opt out of the asset 
    )                       

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

AlgodHTTPError: TransactionPool.Remember: transaction OCHX3FOZGDLA37OPKK2ZP3UGXFWXK3NSU236DXSZPDWNB5IWS2SA: asset 159173586 frozen in MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM

In [57]:
# Open in Algoexplorer to check holders
print('https://testnet.algoexplorer.io/asset/{}'.format(TEMP_id))

https://testnet.algoexplorer.io/asset/159173586


### Destroying the asset
* Use a special `AssetConfigTxn` transaction that has no special roles

In [58]:
# Step 1: prepare
sp = algod_client.suggested_params()

# Step 2: Create special AssetConfigTxn to destroy
txn = AssetConfigTxn(
    sender=Alice['public'],              # Manager must be sender
    sp=sp,
    index=TEMP_id,
    total = None,                        # <---- To destroy, set total to "None"
    strict_empty_address_check=False     # include this option in destroy ops
    )

#Step 2: Send 
stxn = txn.sign(Alice['private'])

# Step 3: Send the transaction and check for errors
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                              # print entire error message
    if ("cannot destroy asset" in str(err)):                # check for specific type of error
        print("Cannot destroy asset {}, not holding all tokens.".format(TEMP_id))  
    if ("does not exist or has been deleted" in str(err)): 
        print("Cannot destroy asset. It does not exist (any more).")    

TransactionPool.Remember: transaction CCSEJ2KPNVOQM5JOR2SGUAMVY6WMXFIISCQTUTLSDPAREO66X5DA: cannot destroy asset: creator is holding only 917/1000
Cannot destroy asset 159173586, not holding all tokens.


In [59]:
# Step 4: Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

Current round is  27733969.
Transaction GK64WXKUJTOJI6VOC6KMQMZFLUUDJZT7P55OXIYEUKPRJ262VONQ confirmed in round 27733957.


{'confirmed-round': 27733957,
 'pool-error': '',
 'txn': {'sig': 'L44uwqBJNJyQGEOfeTdIUc968LjsDPXKTNm9XCGtgjfPA8i5KrhjkWkJ6MS+sw/9n/8f+Cx+PoUw+g5idPNlCg==',
  'txn': {'afrz': True,
   'fadd': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM',
   'faid': 159173586,
   'fee': 1000,
   'fv': 27733955,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 27734955,
   'snd': 'ZFTSGNBBFDH544IDUVB7S4B67CX6PJL6MMFJYT3GGDH2GWJ4N7H6VKRU2Y',
   'type': 'afrz'}}}

In [60]:
# Now check the "assets created" for that account.
account_info = algod_client.account_info(MyAlgo['public'])

In [61]:
account_info['created-assets']

[{'index': 159159432,
  'params': {'clawback': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM',
   'creator': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'decimals': 2,
   'default-frozen': False,
   'freeze': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM',
   'manager': 'GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4',
   'name': 'Husky Token',
   'name-b64': 'SHVza3kgVG9rZW4=',
   'reserve': 'GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4',
   'total': 100000,
   'unit-name': 'HUSKEN',
   'unit-name-b64': 'SFVTS0VO',
   'url': 'www.usi.ch/wsc',
   'url-b64': 'd3d3LnVzaS5jaC93c2M='}},
 {'index': 159171974,
  'params': {'clawback': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM',
   'creator': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA',
   'decimals': 2,
   'default-frozen': False,
   'freeze': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM',
   'manager': 'GFJ3O3QBJ4H3KP

#### EXERCISE
* Check the holdings of MyAlgo, Alice and Bob. 
* Check the asset-id on Algoexplorer.
* Is the asset really gone from the blockchain? discuss

## Things that don't work

* Holding an ASA if you do not hold sufficient ALGOs
    * Minimum 0.1 ALGO per account plus 0.1 ALGO per ASA
* Destroying an ASA if the creator does not hold all coins